## Load libraries and modules

In [ ]:
import tensorflow as tf

In [ ]:
#check the GPU colab assigns to you
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
tf.device('/device:GPU:0')

In [ ]:
import tensorflow as tf
import tensorflow.keras
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)


from tensorflow.keras.models import *
from tensorflow.keras.layers import *

import os
import rasterio
#import rasterio.warp             # Reproject raster samples
from rasterio import windows
#import geopandas as gps
#import PIL.Image
#import PIL.ImageDraw

from osgeo import gdal


%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)
import matplotlib.pyplot as plt
import gc

from pathlib import Path

from itertools import product
from tqdm import tqdm

import geopandas as gpd
import pandas as pd
import os

from matplotlib import pylab
pylab.rcParams['figure.figsize'] = (15, 10)
# !pip install ipython-autotime
# %load_ext autotime

In [ ]:
#set the sys path where the modules locates
import sys
sys.path.insert(0,"core")

#If you are using Google Colaboratory, modify the path here
#sys.path.insert(0,"/content/drive/MyDrive/Colab/zijingwu-Satellite-based-monitoring-of-wildebeest/core")
from preprocess import *
from data_generator import DataGenerator

from model import *

from evaluation import *

from visualization import *

import importlib

from predict import *

## Detect the wildebeest on the test dataset

In [ ]:
%%time
#Calculate the accuracy for each density class
#Criteria: a predicted point is within a searching radius (in meter) of the ground truth point

# data projection: prediction points match the image (4326) -> convert EPSG 32736; true points also 4326 -> convert EPSG 32736
        
        
# annotation_path = "/content/drive/MyDrive/Colab/zijingwu-Satellite-based-monitoring-of-wildebeest/SampleData/Test2023"
annotation_path = "/home/zijing/wildebeest/Sample_test2023/WV3_2023_test_label"

low_grid = os.path.join(annotation_path, "2023 test sample grid/Low_desnity_test_sample_grid_2023.shp")
medium_grid= os.path.join(annotation_path, "2023 test sample grid/Medium_desnity_test_sample_grid_2023.shp")
high_grid = os.path.join(annotation_path, "2023 test sample grid/High_desnity_test_sample_grid_2023.shp")
veryhigh_grid = os.path.join(annotation_path, "2023 test sample grid/Very_high_desnity_test_sample_grid_2023.shp")

low_annotation = os.path.join(annotation_path, "2023 test samples/Low_density_test_samples_2023.shp")
medium_annotation = os.path.join(annotation_path, "2023 test samples/Medium_density_test_samples_2023.shp")
high_annotation = os.path.join(annotation_path, "2023 test samples/High_density_test_samples_2023.shp")
veryhigh_annotation = os.path.join(annotation_path, "2023 test samples/Very_high_density_test_samples_2023.shp")
# cluster_size = 9

item= ["low", "medium","high", "veryhigh" ]
grid_path=[low_grid, medium_grid, high_grid, veryhigh_grid]
label_path=[low_annotation, medium_annotation, high_annotation, veryhigh_annotation]


define_crs = 'EPSG:32736'

# search_distance = 0.84
output_accuracy_path = os.path.join(annotation_path, 'paper_test.csv')


Folder = "/home/zijing/wildebeest"
WEIGHT_PATH = os.path.join(Folder,'tmp/checkpoint/weights')
cluster_size = 16


nfold = 5

NUM = 2
PATCH_SIZE = 336
TILE_MAX_SIZE = PATCH_SIZE * NUM

INPUT_BANDS = [0,1,2]
NUMBER_BANDS=len(INPUT_BANDS)


CONTRAST = False
fold_nums = 5

weight_set = 0.8
lr_set = 0.0001

model = unet(pretrained_weights=None, input_size=(PATCH_SIZE,PATCH_SIZE,NUMBER_BANDS), lr = lr_set, regularizers = regularizers.l2(0.0001))

In [ ]:
#Run prediction on test dataset and save the prediction as points
#The test images are stored separately in the folder for each class
#The predictions will be saved in the corresponding folders (see below Final_Output_dir)

for i in range(len(item)): 

    grids = grid_path[i]
    gt = label_path[i]
    to_eval = item[i]
    print("Class to be evaluated: ", to_eval)
    
    tiled_folder = os.path.join(annotation_path, "test_images/{}/image".format(to_eval))
    Output_dir = os.path.join(annotation_path, "test_images/{}/predict_test".format(to_eval))
    Final_Output_dir = os.path.join(annotation_path, "test_images/{}/predict_test_merge".format(to_eval))

    predict_file_name = '{}_merge.shp'.format(to_eval)

    target_images = get_images_to_predict(tiled_folder)
 
    if not os.path.exists(Output_dir):
        os.makedirs(Output_dir)
                
    for ti in target_images:
        print(ti)
        f = ti
        file_name = os.path.split(f)[1]
        out_name, file_extension = os.path.splitext(file_name)
        print(out_name)
        shp_path = os.path.join(Output_dir, out_name+'.shp')
        mask_path = os.path.join(Output_dir, out_name+'.tif')
    
        if Path(shp_path).is_file() == True:
          print(f"Prediction already exists. Skip.")
          continue
    
        with rasterio.open(f) as src:
    
            detectedMask = detect_wildebeest(model, WEIGHT_PATH, src, width=PATCH_SIZE, height=PATCH_SIZE, stride = 256,
                                batch_size=12, stretch=CONTRAST, num_folds=nfold) # WIDTH and HEIGHT should be the same and in this case Stride is 50 % width
            # visualize_prediction(detectedMask)
            #Write the mask to file
            # visualize_data(np.moveaxis(np.uint8(src.read()), 0,-1),np.expand_dims(detectedMask, axis=2))
            writeResultsToDisk(detectedMask, src, src.meta['transform'], shp_path, None, cluster_size)

        gc.collect()
    

    if not os.path.exists(Final_Output_dir):
        os.makedirs(Final_Output_dir)
    
    file_list = []
    for root, dirs, files in os.walk(Output_dir, topdown=False):
        for name in files:
          out_name, file_extension = os.path.splitext(name)
          if file_extension == '.shp':
            print(name)              
            points = gpd.read_file(os.path.join(root,name))      
            file_list.append(points)
    rdf = gpd.pd.concat(file_list, ignore_index=True)     
    rdf.to_file(os.path.join(Final_Output_dir, predict_file_name))
    print(rdf.count())


In [ ]:
%%time

"""
pixel wise comparison and accuracy evaluation

a predicted point will be considered correct if it falls within the seaching radius (unit: number of pixels) around the ground truth point
"""

cluster_size_list = [16]
search_distance_list = [3]

define_crs = 'EPSG:32736'


Total_TP = 0
Total_FP = 0
Total_FN = 0


df_list = []
wrong_detection_list = []

for cluster_size in cluster_size_list:
    print("Cluster size: ", cluster_size)
    
    for s in search_distance_list:
        print("Search distance: ", s)
        
        all_Total_TP = 0
        all_Total_FP = 0
        all_Total_FN = 0
        
        accuracy_list = []
        
        
        for i in range(len(item)):
                      
            grids = grid_path[i]
            gt = label_path[i]
            to_eval = item[i]
            print("Class to be evaluated: ", to_eval)

            if os.path.exists(grids) != True:
                print(f"{grids} does not exist.")

            # iterate over the smample plots
            roi = gpd.read_file(grids)
            print(roi.crs)

            if roi.crs is None:
                roi = roi.set_crs(define_crs)
            roi = roi[~roi.is_empty] # remove potential empty polygons
            print("found {} valid geometries".format(len(roi)))
    #             print("Type of roi: ", type(roi))
    #             print(roi)
            Total_TP = 0
            Total_FP = 0
            Total_FN = 0
            
#             out_path = os.path.join(annotation_path, 'test_images/{}/predict_combine_{}'.format(to_eval, cluster_size))
            out_path = os.path.join(annotation_path, 'test_images/{}/predict_test_merge'.format(to_eval))
    
            if os.path.exists(out_path) == False:
                print("Prediction point path does not exists!")
#             out_name = '{}_cluster_{}'.format(to_eval, cluster_size)
            out_name = '{}_merge'.format(to_eval)

            shp_path = os.path.join(out_path, out_name+'.shp') 
            
            
            for index in range(len(roi)):
                print(index)
#                 seg_map = os.path.join(seg_map_path, "image_{:0>6d}.tif".format(index))
                img = os.path.join(annotation_path, "test_images/"+to_eval+'/image', "image_{:0>6d}.tif".format(index))
                with rasterio.open(img) as src:
                    bou = src.bounds
                    true_pts = gpd.read_file(gt)
                    predict_pts = gpd.read_file(shp_path)
                    print(true_pts.crs)
                    print(src.crs)
                    if true_pts.crs == src.crs and predict_pts.crs == src.crs:
                        print('The crs of the points are the same as the crs from the raster image!')
                        bou = src.bounds
                        true_pts = gpd.read_file(gt, bbox = bou, encoding='utf-8')
                        predict_pts = gpd.read_file(shp_path, bbox = bou, encoding='utf-8')
              
                    else:    
                        print('The crs of the points are different from the crs from the raster image!')
                        true_pts = gpd.read_file(gt, encoding='utf-8').to_crs(src.crs)
                        predict_pts = gpd.read_file(shp_path, encoding='utf-8').to_crs(src.crs)
                        
                        roi = roi.to_crs(src.crs)
#                         df_bou = gpd.GeoDataFrame(gpd.GeoSeries(bou), columns=['geometry'])
                        true_pts = gpd.sjoin(true_pts, roi[roi.index==index], op='within', how='inner')
                        predict_pts = gpd.sjoin(predict_pts, roi[roi.index==index], op='within', how='inner')
                        
                    print(len(true_pts.index))
                    print(len(predict_pts.index))
                    true_pts_x, true_pts_y = CrsToPixel(true_pts.geometry, src)
                    predict_pts_x, predict_pts_y = CrsToPixel(predict_pts.geometry, src)               
                    
                accuracy = evaluation_pixel(list(zip(true_pts_x,true_pts_y)), list(zip(predict_pts_x,predict_pts_y)), threshold=s)
                print("Accuracy of grid {}".format(index), accuracy)
                if accuracy['FP'] > 10 or accuracy['FN'] > 10:
                    accuracy["index"] = index
                    accuracy["category"] = to_eval
                    accuracy["search_distance"] = s
                    wrong_detection_list.append(accuracy)
                '''    
                if accuracy['FP'] > 10 or accuracy['FN'] > 10:
                    fig, ax = plt.subplots(1,3, sharex=True, sharey=True)
                    ax[0].set_title("Satellite image")
                    ax[1].set_title("Ground truth")
                    ax[2].set_title("Detection")
                    data = cv2.imread(img)
                    ax[0].imshow(data)
                    ax[1].imshow(data)
                    ax[1].scatter(true_pts_y, true_pts_x, color='red')
                    ax[2].imshow(data)
                    ax[2].scatter(predict_pts_y, predict_pts_x, color='blue')                    
                    plt.show()
                '''        
                Total_TP += accuracy['TP']
                Total_FP += accuracy['FP']
                Total_FN += accuracy['FN']

        #         print(len(total_pts))

            Total_precision = Total_TP/(Total_TP+Total_FP+epsilon)
            Total_recall = Total_TP/(Total_TP+Total_FN+epsilon)
            Total_f1 = 2*(Total_precision*Total_recall)/(Total_precision+Total_recall+epsilon)
        #         print("Total Precision: ", Total_precision)
        #         print("Total Recall: ", Total_recall)
        #         print("Total F1-score: ", Total_f1)
            total_accuracy = {
              "TP": Total_TP,
              "FP": Total_FP,
              "FN": Total_FN,
              "Precision":Total_precision,
              "Recall":Total_recall,
              "F1":Total_f1
          }   

            print("Accuracy of class {}".format(to_eval), total_accuracy)
            accuracy_list.append(total_accuracy) 
            
            all_Total_TP += total_accuracy['TP']
            all_Total_FP += total_accuracy['FP']
            all_Total_FN += total_accuracy['FN']

        all_Total_precision = all_Total_TP/(all_Total_TP+all_Total_FP+epsilon)
        all_Total_recall = all_Total_TP/(all_Total_TP+all_Total_FN+epsilon)
        all_Total_f1 = 2*(all_Total_precision*all_Total_recall)/(all_Total_precision+all_Total_recall+epsilon)


        all_accuracy = {
              "TP": all_Total_TP,
              "FP": all_Total_FP,
              "FN": all_Total_FN,
              "Precision":all_Total_precision,
              "Recall":all_Total_recall,
              "F1":all_Total_f1
          }
        print("Accuracy of all classes together: ", all_accuracy)
        item.append("total")
        accuracy_list.append(all_accuracy)
#         print(accuracy_list)            

        df = pd.DataFrame(data=accuracy_list)
#         print(df)
#         print(item)
        df['new_index'] = item
    #     df.set_index('new_index')
        df['cluster size'] = cluster_size
        df['search distance'] = s
#         print(df)
#         df.to_csv(os.path.join(annotation_path, 'accuracy2023_cluster_{}_search_pixel{}.csv'.format(cluster_size, s)), index = False)

        item.pop()
        df_list.append(df)

all_df = pd.concat(df_list)
print(all_df)
all_df.to_csv(os.path.join(annotation_path, 'Final_accuracy_test.csv'), index = False)

wrong_detection = pd.DataFrame(data=wrong_detection_list)
wrong_detection.to_csv(os.path.join(annotation_path, 'Final_accuracy_test_wrong.csv'), index = False)